In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import os

In [2]:
def read_xml(path):
    tree = ET.parse(path)
    root = tree.getroot()
    return root

def get_placemarks(root):
    return root.findall('.//{http://www.opengis.net/kml/2.2}Placemark')

def get_line_strings(placemark):
    return placemark.findall('.//{http://www.opengis.net/kml/2.2}LineString')

In [3]:
# read pavement.xml file
xml_file = 'Pavement.xml'
xml_root = read_xml(xml_file)
placemarks = get_placemarks(xml_root)


In [4]:
linestring_list = []
for placemark in placemarks:
    line_strings = get_line_strings(placemark)
    temp = line_strings[0].findall('.//{http://www.opengis.net/kml/2.2}coordinates')[0].text.split(',0')[:-1]
    linestring_list.append(temp)



In [5]:
def get_Points(placemark):
    return placemark.findall('.//{http://www.opengis.net/kml/2.2}Point')

# <Placemark>
# 	<ExtendedData><SchemaData schemaUrl="#CRASH_CRASH">
# 		<SimpleData name="OBJECTID">2181450</SimpleData>
# 		<SimpleData name="CRASH_KEY">2009000003</SimpleData>
# 		<SimpleData name="CASENUMBER">2009482002</SimpleData>
# 		<SimpleData name="LECASENUM">W09-000195</SimpleData>
# 		<SimpleData name="CRASH_DATE">2009/01/01 00:00:00+00</SimpleData>
# 		<SimpleData name="CRASH_MONTH">1</SimpleData>
# 		<SimpleData name="CRASH_DAY">5</SimpleData>
# 		<SimpleData name="TIMESTR">13:11</SimpleData>
# 		<SimpleData name="DISTRICT">2</SimpleData>
# 		<SimpleData name="COUNTY_NUMBER">7</SimpleData>
# 		<SimpleData name="LITERAL">WILLISTON AVE &amp; W 9TH ST</SimpleData>
# </Placemark>

# get crash date from xml file and get only year
def get_crash_date(placemark):
    return placemark.findall('.//{http://www.opengis.net/kml/2.2}SimpleData')[4].text.split('/')[0]

def get_objectid(placemark):
    return placemark.findall('.//{http://www.opengis.net/kml/2.2}SimpleData')[0].text

def get_crcomnnr(placemark):
    return placemark.findall('.//{http://www.opengis.net/kml/2.2}SimpleData')[13].text
def get_csev(placemark):
    return placemark.findall('.//{http://www.opengis.net/kml/2.2}SimpleData')[23].text

def get_csrfcnd(placemark):
    return placemark.findall('.//{http://www.opengis.net/kml/2.2}SimpleData')[18].text

def get_light(placemark):
    return placemark.findall('.//{http://www.opengis.net/kml/2.2}SimpleData')[17].text


In [6]:
xmk_file_crash = "Crash_Data_2022.xml"
xml_root_crash = read_xml(xmk_file_crash)
placemarks_crash = get_placemarks(xml_root_crash)
print(len(placemarks_crash))


53599


In [7]:
crash_df = pd.read_csv('Crash_Data_(SOR)_2022.csv')
print(len(crash_df))

52976


In [8]:
# pavement_df = pd.read_csv('Pavement.csv')
pavement_df = pd.read_csv('Pavement_with_crashes_2022_cf.csv')

<ipython-input-8-d79aaa7556e2>:2: DtypeWarning: Columns (11,87,99,115,116,119,120,121,122,123,131,132,147,148,150,151,152,153,154,155,156,185,186,187,188,192,193,194,195,196,199,200,201,202,209,210,211,212,215,218,219,225) have mixed types. Specify dtype option on import or set low_memory=False.
  pavement_df = pd.read_csv('Pavement_with_crashes_2022_cf.csv')


In [9]:
points_list = []
i=0
for placemark in placemarks_crash:
    year = get_crash_date(placemark)
    id = get_objectid(placemark)
    points = get_Points(placemark)
    if year == '2022':
        if len(points)==0:
            continue
        x,y,z = points[0].findall('.//{http://www.opengis.net/kml/2.2}coordinates')[0].text.split(',')
        id =int(id)
        x = float(x)
        y = float(y)
        temp =(id,x,y)
        points_list.append(temp)

In [10]:
print(len(points_list))

53599


In [11]:

objectid_list = []
for i in range(len(crash_df)):
    objectid_list.append(crash_df['OBJECTID'][i])
points_id_list = []
for i in range(len(points_list)):
    points_id_list.append(points_list[i][0])
intersection = set(objectid_list).intersection(set(points_id_list))
print(len(intersection))

52930


In [12]:
# use this with tolerance instead of exact match
iri_dict = {}
pci={}
crack={}
fault={}
rut={}
frict={}
crash_freq={}
for i in range(len(points_list)):
    id,x,y = points_list[i]
    for line_string in linestring_list:
        for j in range(len(line_string)):
          # keep some tolerance instead of exact match
            x1,y1 = line_string[j].split(',')
            x1 = float(x1)
            y1 = float(y1)
            if abs(x1-x) < 0.0002 and abs(y1-y) < 0.0002:
                iri_dict[id] = pavement_df['IRI'][j]
                pci[id] = pavement_df['PCI_2'][j]
                crack[id] = pavement_df['CRACK_INDX'][j]
                fault[id] = pavement_df['FAULT_INDX'][j]
                rut[id] = pavement_df['RUT'][j]
                frict[id] = pavement_df['FRICT'][j]
                crash_freq[id] = pavement_df['Crash_Frequency'][j]
                print(i)
                break
            else:
                continue

1
7


In [ ]:
# add iri to crash_df
iri_list = []
pci_list = []
crack_list = []
fault_list = []
rut_list = []
frict_list = []
crash_freq_list = []
for i in range(len(crash_df)):
    id = crash_df['OBJECTID'][i]
    if id in iri_dict.keys():
        iri_list.append(iri_dict[id])
        pci_list.append(pci[id])
        crack_list.append(crack[id])
        fault_list.append(fault[id])
        rut_list.append(rut[id])
        frict_list.append(frict[id])
        crash_freq_list.append(crash_freq[id])

    else:
        iri_list.append('NA')
        pci_list.append('NA')
        crack_list.append('NA')
        fault_list.append('NA')
        rut_list.append('NA')
        frict_list.append('NA')
        crash_freq_list.append('NA')


crash_df['IRI'] = iri_list
crash_df['PCI_2'] = pci_list
crash_df['CRACK_INDX'] = crack_list
crash_df['FAULT_INDX'] = fault_list
crash_df['RUT'] = rut_list
crash_df['FRICT'] = frict_list
crash_df['Crash_Frequency'] = crash_freq_list
crash_df.to_csv('Crash_Data_(SOR)_2022_IRI_others.csv',index=False)